In [1]:
import pandas as pd
import gzip

In [2]:
from collections import defaultdict
mane_gff = '/common/zhangz2lab/shared/genomes/gff/MANE.GRCh38.v1.1.ensembl_genomic.gff.gz'

def read_mane(fp):
    cds_regions_loaded = defaultdict(list)
    with gzip.open(fp, 'rt') as f:  # Open the GFF3 file
        for line in f:
            if not line.startswith('#'):  # Skip header lines
                parts = line.strip().split('\t')
                if not parts[0] in ['chr%i'%i for i in range(1,23)] + ['chrX', 'chrY']:
                    continue
                if parts[2] == 'CDS':  # Focus on CDS entries
                    attributes = parts[8]
                    gene_id = [attr for attr in attributes.split(';') if 'gene_name' in attr][0].split('=')[1]
                    strand = parts[6]
                    start = parts[3]
                    end = parts[4]
                    reading_frame = parts[7]
                    chrom = parts[0]
                    cds_regions_loaded[gene_id].append((int(start), int(end), int(reading_frame), strand, chrom))
                    
    return cds_regions_loaded

cds_regions_loaded = read_mane(mane_gff)

In [3]:
print(cds_regions_loaded.get("PRDM16"))  # Example usage


[(3069260, 3069296, 0, '+', 'chr1'), (3186125, 3186474, 2, '+', 'chr1'), (3244087, 3244137, 0, '+', 'chr1'), (3385152, 3385286, 0, '+', 'chr1'), (3396491, 3396593, 0, '+', 'chr1'), (3402791, 3402998, 2, '+', 'chr1'), (3404739, 3404886, 1, '+', 'chr1'), (3405495, 3405648, 0, '+', 'chr1'), (3411384, 3412800, 2, '+', 'chr1'), (3414560, 3414647, 1, '+', 'chr1'), (3417828, 3417997, 0, '+', 'chr1'), (3418667, 3418744, 1, '+', 'chr1'), (3425581, 3425750, 1, '+', 'chr1'), (3426051, 3426225, 2, '+', 'chr1'), (3430872, 3431108, 1, '+', 'chr1'), (3431966, 3432140, 1, '+', 'chr1'), (3433677, 3433811, 0, '+', 'chr1')]


In [4]:
def parse_string_to_dict(info_string):
    """Convert a VCF INFO field string to a dictionary."""
    info_dict = {}
    for item in info_string.split(';'):
        if '=' in item:
            key, value = item.split('=', 1)
            info_dict[key] = value
    return info_dict

def parse_clinvar_to_dataframe(filepath):
    with gzip.open(filepath, 'rt') as file:
        data = []  # List to store combined information for each variant
        for line in file:
            if line.startswith('#'):
                continue  # Skip header lines
            fields = line.strip().split('\t')
            info_field = fields[7]  # INFO field is the 8th field in VCF format
            info_dict = parse_string_to_dict(info_field)
            
            # Combine first 7 fields with extracted INFO data
            variant_info = fields[:7] + [
                info_dict.get('CLNDN'),
                info_dict.get('CLNDISDB'),
                info_dict.get('CLNSIG'),
                info_dict.get('GENEINFO'),
                info_dict.get('CLNREVSTAT'),
                info_dict.get('CLNHGVS'),
                info_dict.get('CLNDISDBINCL'),
                info_dict.get('CLNVI'),
                info_dict.get('AF_EXAC'),
                info_dict.get('AF_ESP'),
                info_dict.get('AF_TGP'),
                info_dict.get('MC'),
            ]
            
            data.append(variant_info)
        
        # Convert the accumulated data into a pandas DataFrame
        columns = ['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER',
                   'cln_disease', 'cln_disease_id', 'cln_significance', 'cln_gene',
                   'cln_review', 'cln_hgvs', 'cln_disease_dbs', 'cln_clnvi',
                   'cln_af_exac', 'cln_af_esp', 'cln_af_tgp', 'cln_molecular_consequence']
        df = pd.DataFrame(data, columns=columns)
        
        return df

# Example usage
filepath = '/common/zhangz2lab/fzzhang/knowledge_graphs/wen-chyu/data/variant/clinvar/clinvar.vcf.gz' 
df = parse_clinvar_to_dataframe(filepath)

df = df.dropna(subset=['cln_gene'])
df['gene_name'] = df.apply(lambda row: row['cln_gene'].split(':')[0], axis=1)

# Display the first few rows of the DataFrame
df.head()

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,cln_disease,cln_disease_id,cln_significance,cln_gene,cln_review,cln_hgvs,cln_disease_dbs,cln_clnvi,cln_af_exac,cln_af_esp,cln_af_tgp,cln_molecular_consequence,gene_name
0,1,925952,1019397,G,A,.,.,not_provided,MedGen:CN517202,Uncertain_significance,SAMD11:148398,"criteria_provided,_single_submitter",NC_000001.11:g.925952G>A,None,None,None,None,None,SO:0001583|missense_variant,SAMD11
1,1,925956,1543320,C,T,.,.,not_provided,MedGen:CN517202,Likely_benign,SAMD11:148398,"criteria_provided,_single_submitter",NC_000001.11:g.925956C>T,None,None,None,None,None,SO:0001819|synonymous_variant,SAMD11
2,1,925969,1648427,C,T,.,.,not_provided,MedGen:CN517202,Likely_benign,SAMD11:148398,"criteria_provided,_single_submitter",NC_000001.11:g.925969C>T,None,None,None,None,None,SO:0001583|missense_variant,SAMD11
3,1,925976,1362713,T,C,.,.,not_provided,MedGen:CN517202,Uncertain_significance,SAMD11:148398,"criteria_provided,_single_submitter",NC_000001.11:g.925976T>C,None,None,None,None,None,SO:0001583|missense_variant,SAMD11
4,1,925986,1568423,C,T,.,.,not_provided,MedGen:CN517202,Likely_benign,SAMD11:148398,"criteria_provided,_single_submitter",NC_000001.11:g.925986C>T,None,None,None,None,None,SO:0001819|synonymous_variant,SAMD11


In [5]:
filtered_df = df[
    (df['cln_disease'].str.contains('arrhythmia', case=False, na=False)) & 
     (df['cln_molecular_consequence'].str.contains('missense', case=False, na=False))
]


# Display the filtered DataFrame
filtered_df.head()


,CHROM,POS,ID,REF,ALT,QUAL,FILTER,cln_disease,cln_disease_id,cln_significance,cln_gene,cln_review,cln_hgvs,cln_disease_dbs,cln_clnvi,cln_af_exac,cln_af_esp,cln_af_tgp,cln_molecular_consequence,gene_name
65750,1,115726998,35772,T,C,.,.,Cardiac_arrhythmia|Cardiomyopathy|Catecholamin...,"EFO:EFO_0004269,Human_Phenotype_Ontology:HP:00...",Benign/Likely_benign,CASQ2:845,"criteria_provided,_multiple_submitters,_no_con...",NC_000001.11:g.115726998T>C,MedGen:CN169374,"HGMD:CM106915|ARUP_Laboratories,_Molecular_Gen...",0.00829,0.02660,0.02835,SO:0001583|missense_variant,CASQ2
85241,1,169130013,35698,C,T,.,.,Cardiac_arrhythmia,"EFO:EFO_0004269,Human_Phenotype_Ontology:HP:00...",Uncertain_significance,ATP1B1:481|NME7:29922,no_assertion_criteria_provided,NC_000001.11:g.169130013C>T,None,None,None,None,None,SO:0001583|missense_variant,ATP1B1
122394,1,237270595,178876,G,C,.,.,Cardiac_arrhythmia|Cardiomyopathy|Catecholamin...,"EFO:EFO_0004269,Human_Phenotype_Ontology:HP:00...",Uncertain_significance,RYR2:6262,"criteria_provided,_multiple_submitters,_no_con...",NC_000001.11:g.237270595G>C,None,None,None,None,None,SO:0001583|missense_variant,RYR2
123625,1,237550628,201235,C,T,.,.,Cardiomyopathy|Arrhythmogenic_right_ventricula...,"Human_Phenotype_Ontology:HP:0001638,MONDO:MOND...",Conflicting_interpretations_of_pathogenicity,RYR2:6262,"criteria_provided,_conflicting_interpretations",NC_000001.11:g.237550628C>T,None,None,0.00017,None,None,SO:0001583|missense_variant,RYR2
123665,1,237566603,36740,G,A,.,.,Primary_dilated_cardiomyopathy|Cardiac_arrhyth...,"EFO:EFO_0000407,Human_Phenotype_Ontology:HP:00...",Conflicting_interpretations_of_pathogenicity,RYR2:6262,"criteria_provided,_conflicting_interpretations",NC_000001.11:g.237566603G>A,None,None,0.00015,None,None,SO:0001583|missense_variant,RYR2


In [6]:
new_bed_df = pd.read_csv('output_bed_arm.csv', index_col=0)
# Remove 'chr' prefix from 'chrom' column in new_bed_df
new_bed_df['chrom'] = new_bed_df['chrom'].str.replace('chr', '')
print(new_bed_df)
# Convert 'chrom' columns to strings in both DataFrames if they are not already
filtered_df['CHROM'] = filtered_df['CHROM'].astype(str)
new_bed_df['chrom'] = new_bed_df['chrom'].astype(str)

# Ensure 'POS' in filtered_df and 'end' in new_bed_df are integers
filtered_df['POS'] = filtered_df['POS'].astype(int)
new_bed_df['end'] = new_bed_df['end'].astype(int)

# Attempt the merge again
matches = filtered_df.merge(new_bed_df, left_on=['CHROM', 'POS'], right_on=['chrom', 'end'])
print(matches)
print(matches['REF'])

# Perform the merge to identify rows to keep ("left_only") and use suffixes to handle overlapping column names
# We'll use suffixes that are unlikely to cause conflicts and are easy to filter out
merged_df = pd.merge(filtered_df, matches, on=['CHROM', 'POS'], how='left', indicator=True, suffixes=('', '_drop'))

# Filter to keep only rows from filtered_df that don't have matches in new_bed_df
filtered_df_no_matches = merged_df[merged_df['_merge'] == 'left_only']

# Now, remove columns that were added from 'matches' (those with '_drop' suffix) and the '_merge' column
# This is done by filtering out columns that end with '_drop' and dropping the '_merge' column
filtered_df_no_matches = filtered_df_no_matches.loc[:, ~filtered_df_no_matches.columns.str.endswith('_drop')].drop(columns=['_merge'])


filtered_df = filtered_df_no_matches 
# Now, filtered_df_no_matches should have the original structure without "_x" or "_y" suffixes
print(filtered_df_no_matches.head())



    chrom      start        end         original_position  score
0      11    2445116    2445117     chr11:2466347-2466347      1
1      11    2445418    2445419     chr11:2466649-2466649      1
2      11    2445446    2445447     chr11:2466677-2466677      1
3      11    2445447    2445448     chr11:2466678-2466678      1
4      11    2445470    2445471     chr11:2466701-2466701      1
..    ...        ...        ...                       ...    ...
303     3   38575394   38575395    chr3:38616886-38616886      1
304     7  150950274  150950275  chr7:150647363-150647363      1
305    12    2585890    2585891     chr12:2695057-2695057      1
306    12    2567571    2567572     chr12:2676738-2676738      1
307     3   38613771   38613772    chr3:38655263-38655263      1

[308 rows x 5 columns]
    CHROM       POS       ID REF ALT QUAL FILTER  \
0       3  38550338    68027   G   A    .      .   
1       3  38550409    68020   A   C    .      .   
2       3  38550499    68015   C   T    

/tmp/ipykernel_3730043/1895868383.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['CHROM'] = filtered_df['CHROM'].astype(str)
/tmp/ipykernel_3730043/1895868383.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['POS'] = filtered_df['POS'].astype(int)


In [8]:
# Filter the DataFrame to include only rows with 'cln_significance' containing "benign" or "pathogenic"
filtered_df = filtered_df[filtered_df['cln_significance'].str.contains('uncertain', case=False)]

# Filter out conflicting annotations
filtered_df = filtered_df[~filtered_df['cln_significance'].str.contains('conflicting', case=False)]


# Count rows where 'cln_significance' contains "benign"
#benign_count = filtered_df[filtered_df['cln_significance'].str.contains('benign', case=False)].shape[0]

# Assuming you want to count rows where 'cln_significance' contains "pathogenic" (regardless of 'cln_molecular_consequence')
#pathogenic_count = filtered_df[filtered_df['cln_significance'].str.contains('pathogenic', case=False)].shape[0]

#print(f"Number of 'benign' rows: {benign_count}")
#print(f"Number of 'pathogenic' rows: {pathogenic_count}")
filtered_df

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,cln_disease,cln_disease_id,cln_significance,...,cln_af_exac,cln_af_esp,cln_af_tgp,cln_molecular_consequence,gene_name,chrom,start,end,original_position,score
1,1,169130013,35698,C,T,.,.,Cardiac_arrhythmia,"EFO:EFO_0004269,Human_Phenotype_Ontology:HP:00...",Uncertain_significance,...,None,None,None,SO:0001583|missense_variant,ATP1B1,NaN,NaN,NaN,NaN,NaN
2,1,237270595,178876,G,C,.,.,Cardiac_arrhythmia|Cardiomyopathy|Catecholamin...,"EFO:EFO_0004269,Human_Phenotype_Ontology:HP:00...",Uncertain_significance,...,None,None,None,SO:0001583|missense_variant,RYR2,NaN,NaN,NaN,NaN,NaN
6,1,237631470,1691305,A,C,.,.,Arrhythmogenic_right_ventricular_dysplasia_2|C...,"MONDO:MONDO:0010975,MedGen:C1832931,OMIM:60099...",Uncertain_significance,...,None,None,None,SO:0001583|missense_variant,RYR2,NaN,NaN,NaN,NaN,NaN
9,1,237784793,201343,G,A,.,.,Catecholaminergic_polymorphic_ventricular_tach...,"MONDO:MONDO:0017990,MedGen:C5574922,OMIM:PS604...",Uncertain_significance,...,None,None,None,SO:0001583|missense_variant,RYR2,NaN,NaN,NaN,NaN,NaN
10,1,237795312,852605,A,C,.,.,Catecholaminergic_polymorphic_ventricular_tach...,"MONDO:MONDO:0017990,MedGen:C5574922,OMIM:PS604...",Uncertain_significance,...,None,None,None,SO:0001583|missense_variant,RYR2,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,22,20052582,1339009,G,A,.,.,Recurrent_metabolic_encephalomyopathic_crises-...,"MONDO:MONDO:0018820,MedGen:C5567524,OMIM:61687...",Uncertain_significance,...,None,None,None,"SO:0001583|missense_variant,SO:0001619|non-cod...",TANGO2,NaN,NaN,NaN,NaN,NaN
1346,22,20052584,1312498,G,T,.,.,Recurrent_metabolic_encephalomyopathic_crises-...,"MONDO:MONDO:0018820,MedGen:C5567524,OMIM:61687...",Uncertain_significance,...,None,None,None,"SO:0001583|missense_variant,SO:0001619|non-cod...",TANGO2,NaN,NaN,NaN,NaN,NaN
1347,22,20053530,1709923,A,G,.,.,Recurrent_metabolic_encephalomyopathic_crises-...,"MONDO:MONDO:0018820,MedGen:C5567524,OMIM:61687...",Uncertain_significance,...,None,None,None,"SO:0001583|missense_variant,SO:0001619|non-cod...",TANGO2,NaN,NaN,NaN,NaN,NaN
1348,22,20055992,1696470,G,A,.,.,Recurrent_metabolic_encephalomyopathic_crises-...,"MONDO:MONDO:0018820,MedGen:C5567524,OMIM:61687...",Uncertain_significance,...,None,None,None,"SO:0001583|missense_variant,SO:0001619|non-cod...",TANGO2,NaN,NaN,NaN,NaN,NaN


In [9]:
# Get unique CHROM values from the DataFrame
unique_chrom_values = filtered_df['CHROM'].unique()

# Print all unique CHROM values
print("Unique CHROM values in the DataFrame:")
for chrom in unique_chrom_values:
    print(chrom)

# Count the number of unique CHROM values
number_of_unique_chrom_values = len(unique_chrom_values)
print(f"Total number of unique CHROM values: {number_of_unique_chrom_values}")
filtered_df = filtered_df[~filtered_df['CHROM'].isin(['MT', 'NW_009646201.1'])]

Unique CHROM values in the DataFrame:
1
3
4
7
9
11
12
14
15
17
22
Total number of unique CHROM values: 11


In [10]:
from Bio.Seq import Seq

# Function to translate DNA sequence to protein sequence
def translate_dna_to_protein(dna_sequence):
    dna_seq = Seq(dna_sequence)
    protein_seq = dna_seq.translate(to_stop=True)  # to_stop=True will stop at the first stop codon
    return str(protein_seq)

In [11]:
# Get proteins for each gene
from pyfaidx import Fasta
from Bio.Seq import Seq
from tqdm import tqdm

reference_genome = Fasta('/common/zhangz2lab/zhanh/Clinvar/hg38.fa')

def reverse_complement(seq):
    complement = {'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G'}
    return ''.join(complement.get(base, base) for base in reversed(seq))


codon_table = {
    'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M',
    'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T',
    'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K',
    'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',
    'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L',
    'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P',
    'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q',
    'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R',
    'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V',
    'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A',
    'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E',
    'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G',
    'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S',
    'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L',
    'TAC':'Y', 'TAT':'Y', 'TAA':'_', 'TAG':'_',
    'TGC':'C', 'TGT':'C', 'TGA':'_', 'TGG':'W',
}

def translate_dna(sequence):
    protein = ""
    for i in range(0, len(sequence), 3):
        codon = sequence[i:i+3]
        if len(codon) == 3:
            protein += codon_table.get(codon, 'X')  # 'X' for unknown codons
    return protein


In [12]:
gene = 'CASQ2'
dna = ''

for cds_start, cds_end, _, strand, chromosome in cds_regions_loaded[gene]:
    if strand == '+':
        dna += reference_genome[chromosome][cds_start-1:cds_end].seq
    else:
        dna += reference_genome[chromosome][cds_start-1:cds_end].reverse.complement.seq

# why different? Biopython's translation matches UCSC genome browser
strand, translate_dna(dna), translate_dna_to_protein(dna)

('-',
 'MKRTHLFIVGIYFLSSCRAEEGLNFPTYDGKDRVVSLSEKNFKQVLKKYDLLCLYYHEPVSSDKVTQKQFQLKEIVLELVAQVLEHKAIGFVMVDAKKEAKLAKKLGFDEEGSLYILKGDRTIEFDGEFAADVLVEFLLDLIEDPVEIISSKLEVQAFERIEDYIKLIGFFKSEDSEYYKAFEEAAEHFQPYIKFFATFDKGVAKKLSLKMNEVDFYEPFMDEPIAIPNKPYTEEELVEFVKEHQRPTLRRLRPEEMFETWEDDLNGIHIVAFAEKSDPDGYEFLEILKQVARDNTDNPDLSILWIDPDDFPLLVAYWEKTFKIDLFRPQIGVVNVTDADSVWMEIPDDDDLPTAEELEDWIEDVLSGXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX_',
 'MKRTHLFIVGIYFLSSCRAEEGLNFPTYDGKDRVVSLSEKNFKQVLKKYDLLCLYYHEPVSSDKVTQKQFQLKEIVLELVAQVLEHKAIGFVMVDAKKEAKLAKKLGFDEEGSLYILKGDRTIEFDGEFAADVLVEFLLDLIEDPVEIISSKLEVQAFERIEDYIKLIGFFKSEDSEYYKAFEEAAEHFQPYIKFFATFDKGVAKKLSLKMNEVDFYEPFMDEPIAIPNKPYTEEELVEFVKEHQRPTLRRLRPEEMFETWEDDLNGIHIVAFAEKSDPDGYEFLEILKQVARDNTDNPDLSILWIDPDDFPLLVAYWEKTFKIDLFRPQIGVVNVTDADSVWMEIPDDDDLPTAEELEDWIEDVLSGKINTEDDDEDDDDDDNSDEEDNDDSDDDDDE')

In [13]:
flank_amount = 500

sequences = []

for index, variant in filtered_df.iterrows():
    gene_name = variant['gene_name']
    chromosome = "chr" + str(variant['CHROM'])
    position = int(variant['POS'])
    ref_allele = variant['REF']
    alt_allele = variant['ALT']
    label = variant['cln_significance']

    cds_list = cds_regions_loaded[gene_name]  # Get the list of CDS regions for the gene
    cds_list_sorted = sorted(cds_list, key=lambda x: x[0])  # Ensure the CDS regions are sorted


    dna_pos = 0
    is_within_cds = False
    # Find the index of the CDS region that contains the variant
    # cds_regions_loaded is sorted ascending for + strand, and descending for - strand; no need to re-sort
    for i, cds_interval in enumerate(cds_regions_loaded[gene_name]):
        cds_start = cds_interval[0]; cds_end = cds_interval[1]; strand = cds_interval[-2]
        #print(gene_name, dna_pos, position, cds_interval)
        if cds_start <= position <= cds_end:
            if strand == '+':
                dna_pos += position - cds_start
            else:
                dna_pos += cds_end - position
            is_within_cds = True
            break
        dna_pos += cds_end - cds_start + 1
    
    prot_pos = dna_pos//3

    if not is_within_cds:
        continue

    dna = ''
    for cds_start, cds_end, _, strand, chromosome in cds_regions_loaded[gene_name]:
        if strand == '+':
            dna += reference_genome[chromosome][cds_start-1:cds_end].seq
        else:
            dna += reference_genome[chromosome][cds_start-1:cds_end].reverse.complement.seq

    ref_allele_len = len(ref_allele)
    if strand == "+":
        is_ref_match = int(dna[dna_pos:(dna_pos+ref_allele_len)] == ref_allele)
        ref_dna = dna[:dna_pos] + ref_allele + dna[dna_pos+ref_allele_len:]
        alt_dna = dna[:dna_pos] + alt_allele + dna[dna_pos+ref_allele_len:]
    else:
        ref_allele = reverse_complement(ref_allele)
        alt_allele = reverse_complement(alt_allele)
        is_ref_match = int(dna[dna_pos:(dna_pos+ref_allele_len)] == ref_allele)
        ref_dna = dna[:dna_pos] + ref_allele + dna[dna_pos+ref_allele_len:]
        alt_dna = dna[:dna_pos] + alt_allele + dna[dna_pos+ref_allele_len:]

    ref_prot = translate_dna_to_protein(ref_dna)
    alt_prot = translate_dna_to_protein(alt_dna)
    ref_prot_ = ref_prot[max(0, prot_pos-flank_amount):min(len(ref_prot), prot_pos+flank_amount)]
    alt_prot_ = alt_prot[max(0, prot_pos-flank_amount):min(len(alt_prot), prot_pos+flank_amount)]
    ref_dna_ = ref_dna[max(0, dna_pos-flank_amount*3):min(len(ref_dna), dna_pos+flank_amount*3)]
    alt_dna_ = alt_dna[max(0, dna_pos-flank_amount*3):min(len(alt_dna), dna_pos+flank_amount*3)]

    sequences.append({
        'ref_prot': ref_prot_,
        'alt_prot': alt_prot_,
        'ref_dna': ref_dna_,
        'alt_dna': alt_dna_,
        'strand': strand,
        'label': label,
        'gene': gene_name,
        'ref': ref_allele,
        'alt': alt_allele,
        'POS': position,
        'is_ref_match': is_ref_match,
        'molecular_consequences': variant['cln_molecular_consequence']
    })

sequences_df = pd.DataFrame(sequences)



In [14]:
sequences_df.strand.value_counts()

-    664
+    221
Name: strand, dtype: int64

In [15]:
# no ref_allele mismatch
sequences_df[sequences_df.is_ref_match==0]

,ref_prot,alt_prot,ref_dna,alt_dna,strand,label,gene,ref,alt,POS,is_ref_match,molecular_consequences
22,LFIGVIIDNFNQQKKKLGGQDIFMTEEQKKYYNAMKKLGSKKPQKP...,LFIGVIIDNFNQQKKKLGGQDIFMTEEQKKYYNAMKKLGSKKPQKP...,TCTTTATTGGTGTCATCATTGACAACTTCAACCAACAGAAGAAAAA...,TCTTTATTGGTGTCATCATTGACAACTTCAACCAACAGAAGAAAAA...,-,Uncertain_significance,SCN5A,C,T,38550481,0,SO:0001583|missense_variant
64,VGAIPSIMNVLLVCLIFWLIFSIMGVNLFAGKFGRCINQTEGDLPL...,VGAIPSIMNVLLVCLIFWLIFSIMGVNLFAGKFGRCINQTEGDLPL...,GTGGGCGCCATCCCGTCCATCATGAACGTCCTCCTCGTCTGCCTCA...,GTGGGCGCCATCCCGTCCATCATGAACGTCCTCCTCGTCTGCCTCA...,-,Uncertain_significance,SCN5A,G,A,38550890,0,SO:0001583|missense_variant
65,ALVGAIPSIMNVLLVCLIFWLIFSIMGVNLFAGKFGRCINQTEGDL...,ALVGAIPSIMNVLLVCLIFWLIFSIMGVNLFAGKFGRCINQTEGDL...,CCCTGGTGGGCGCCATCCCGTCCATCATGAACGTCCTCCTCGTCTG...,CCCTGGTGGGCGCCATCCCGTCCATCATGAACGTCCTCCTCGTCTG...,-,Uncertain_significance,SCN5A,G,A,38550895,0,SO:0001583|missense_variant
66,VVNALVGAIPSIMNVLLVCLIFWLIFSIMGVNLFAGKFGRCINQTE...,VVNALVGAIPSIMNVLLVCLIFWLIFSIMGVNLFAGKFGRCINQTE...,GTGGTCAATGCCCTGGTGGGCGCCATCCCGTCCATCATGAACGTCC...,GTGGTCAATGCCCTGGTGGGCGCCATCCCGTCCATCATGAACGTCC...,-,Uncertain_significance,SCN5A,G,A,38550905,0,SO:0001583|missense_variant
67,MRVVVNALVGAIPSIMNVLLVCLIFWLIFSIMGVNLFAGKFGRCIN...,MRVVVNALVGAIPSIMNVLLVCLIFWLIFSIMGVNLFAGKFGRCIN...,ATGAGGGTGGTGGTCAATGCCCTGGTGGGCGCCATCCCGTCCATCA...,ATGAGGGTGGTGGTCAATGCCCTGGTGGGCGCCATCCCGTCCATCA...,-,Uncertain_significance,SCN5A,TG,CA,38550914,0,SO:0001583|missense_variant
68,GMRVVVNALVGAIPSIMNVLLVCLIFWLIFSIMGVNLFAGKFGRCI...,GMRVVVNALVGAIPSIMNVLLVCLIFWLIFSIMGVNLFAGKFGRCI...,CATGAGGGTGGTGGTCAATGCCCTGGTGGGCGCCATCCCGTCCATC...,CATGAGGGTGGTGGTCAATGCCCTGGTGGGCGCCATCCCGTCCATC...,-,Uncertain_significance,SCN5A,GAT,AAC,38550915,0,SO:0001583|missense_variant
69,LRTLRALRPLRALSRFEGMRVVVNALVGAIPSIMNVLLVCLIFWLI...,LRTLRALRPLRALSRFEGMRVVVNALVGAIPSIMNVLLVCLIFWLI...,GCGGACGCTGCGTGCACTCCGTCCTCTGAGAGCTCTGTCACGATTT...,GCGGACGCTGCGTGCACTCCGTCCTCTGAGAGCTCTGTCACGATTT...,-,Uncertain_significance,SCN5A,C,G,38550966,0,SO:0001583|missense_variant
70,SLRTLRALRPLRALSRFEGMRVVVNALVGAIPSIMNVLLVCLIFWL...,SLRTLRALRPLRALSRFEGMRVVVNALVGAIPSIMNVLLVCLIFWL...,CACTGCGGACGCTGCGTGCACTCCGTCCTCTGAGAGCTCTGTCACG...,CACTGCGGACGCTGCGTGCACTCCGTCCTCTGAGAGCTCTGTCACG...,-,Uncertain_significance,SCN5A,T,C,38550970,0,SO:0001583|missense_variant
75,WVAYGFKKYFTNAWCWLDFLIVDVSLVSLVANTLGFAEMGPIKSLR...,WVAYGFKKYFTNAWCWLDFLIVDVSLVSLVANTLGFAEMGPIKSLR...,TGGGTGGCCTACGGCTTCAAGAAGTACTTCACCAATGCCTGGTGCT...,TGGGTGGCCTACGGCTTCAAGAAGTACTTCACCAATGCCTGGTGCT...,-,Uncertain_significance,SCN5A,A,G,38551100,0,SO:0001583|missense_variant
195,HLLRPVMLEHPPDTTTPSEEPGGPQMLTSQAPCVDGFEEPGARQRA...,HLLRPVMLEHPPDTTTPSEEPGGPQMLTSQAPCVDGFEEPGARQRA...,CCTCCTCCGCCCTGTGATGCTAGAGCACCCGCCAGACACGACCACG...,CCTCCTCCGCCCTGTGATGCTAGAGCACCCGCCAGACACGACCACG...,-,Uncertain_significance,SCN5A,AG,CC,38579373,0,"SO:0001583|missense_variant,SO:0001627|intron_..."


In [16]:
# ref_prot != alt_prot unless other molecular_consequences exist
sequences_df[sequences_df.ref_prot==sequences_df.alt_prot][['label','molecular_consequences']]

,label,molecular_consequences


In [23]:
sequences_df['gene']
# Assuming sequences_df is your DataFrame and 'gene' is the column you want to save
sequences_df['gene'].to_csv('clinvar_arm.csv', index=False)
df = pd.read_csv("/common/zhangz2lab/zhanh/Propath-2/PLLR_ARM.csv")
df['gene'] = sequences_df['gene']
df['label'] = sequences_df['label']
# For the DataFrame loaded from CSV
print(df.index)

# For the sequences_df DataFrame
print(sequences_df.index)

df.to_csv('/common/zhangz2lab/zhanh/Propath-2/PLLR_ARM.csv', index=False)


RangeIndex(start=0, stop=98, step=1)
RangeIndex(start=0, stop=98, step=1)


In [17]:
# save
sequences_df.to_csv('/common/zhangz2lab/zhanh/esm-variants/cropped/Clinvar_ARM_uncertain_protein.csv', index=False)

In [17]:
print(sequences_df['ref_dna'].iloc[0])
print(sequences_df['alt_dna'].iloc[0])

ATGAAGAGAACTCACTTGTTTATTGTGGGGATTTATTTTCTGTCCTCTTGCAGGGCAGAAGAGGGGCTTAATTTCCCCACATATGATGGGAAGGACCGAGTGGTAAGTCTTTCCGAGAAGAACTTCAAGCAGGTTTTAAAGAAATATGACTTGCTTTGCCTCTACTACCATGAGCCGGTGTCTTCAGATAAGGTCACGCAAAAACAGTTCCAACTGAAAGAAATCGTGCTTGAGCTTGTGGCCCAGGTCCTTGAACATAAAGCTATAGGCTTTGTGATGGTGGATGCCAAGAAAGAAGCCAAGCTTGCCAAGAAACTGGGTTTTGATGAAGAAGGAAGCCTGTATATTCTTAAGGGTGATCGCACAATAGAGTTTGATGGCGAGTTTGCAGCTGATGTCTTGGTGGAGTTCCTCTTGGATCTAATTGAAGACCCAGTGGAGATCATCAGCAGCAAACTGGAAGTCCAAGCCTTCGAACGCATTGAAGACTACATCAAACTCATTGGCTTTTTCAAGAGTGAGGACTCAGAATACTACAAGGCTTTTGAAGAAGCAGCTGAACACTTCCAGCCTTACATCAAATTCTTTGCCACCTTTGACAAAGGGGTTGCAAAGAAATTATCTTTGAAGATGAATGAGGTTGACTTCTATGAGCCATTTATGGATGAGCCCATTGCCATCCCCAACAAACCTTACACAGAAGAGGAGCTGGTGGAGTTTGTGAAGGAACACCAAAGACCCACTCTACGTCGCCTGCGCCCAGAAGAAATGTTTGAAACATGGGAAGATGATTTGAATGGGATCCACATTGTGGCCTTTGCAGAGAAGAGTGATCCAGATGGCTACGAATTCCTGGAGATCCTGAAACAGGTTGCCCGGGACAATACTGACAACCCCGATCTGAGCATCCTGTGGATCGACCCGGACGACTTTCCTCTGCTCGTTGCCTACTGGGAGAAGACTTTCAAGATTGACCTATTCAGGCCACAGATTGGGGTGG

In [18]:
sequences_df['ref_prot'].iloc[0] == sequences_df['alt_prot'].iloc[0]

False

In [19]:
sequences_df

,ref_prot,alt_prot,ref_dna,alt_dna,strand,label,gene,ref,alt,POS,is_ref_match,molecular_consequences
0,MKRTHLFIVGIYFLSSCRAEEGLNFPTYDGKDRVVSLSEKNFKQVL...,MKRTHLFIVGIYFLSSCRAEEGLNFPTYDGKDRVVSLSEKNFKQVL...,ATGAAGAGAACTCACTTGTTTATTGTGGGGATTTATTTTCTGTCCT...,ATGAAGAGAACTCACTTGTTTATTGTGGGGATTTATTTTCTGTCCT...,-,Benign/Likely_benign,CASQ2,A,G,115726998,1,SO:0001583|missense_variant
1,RFSLKDTEDEVRDIIRSNIHLQGKLEDPAIRWQMALYKDLPNRTDD...,RFSLKDTEDEVRDIIRSNIHLQGKLEDPAIRWQMALYKDLPNRTDD...,CGATTTAGCCTGAAAGATACCGAGGATGAAGTACGAGATATAATCC...,CGATTTAGCCTGAAAGATACCGAGGATGAAGTACGAGATATAATCC...,+,Pathogenic,RYR2,A,C,237783695,1,SO:0001583|missense_variant
2,AKSCHDEEDDDGEEEVKSFEEKEMEKQKLLYQQARLHDRGAAEMVL...,AKSCHDEEDDDGEEEVKSFEEKEMEKQKLLYQQARLHDRGAAEMVL...,CAAAGAGTTGTCATGATGAGGAAGATGACGATGGTGAAGAGGAAGT...,CAAAGAGTTGTCATGATGAGGAAGATGACGATGGTGAAGAGGAAGT...,+,Pathogenic,RYR2,C,T,237784299,1,SO:0001583|missense_variant
3,KKMTVKDMVTAFFSSYWSIFMTLLHFVASVFRGFFRIICSLLLGGS...,KKMTVKDMVTAFFSSYWSIFMTLLHFVASVFRGFFRIICSLLLGGS...,AAAAAGATGACCGTGAAGGACATGGTCACGGCCTTCTTTTCATCCT...,AAAAAGATGACCGTGAAGGACATGGTCACGGCCTTCTTTTCATCCT...,+,Likely_pathogenic,RYR2,G,A,237808916,1,SO:0001583|missense_variant
4,SEDLTDLKELTEESDLLSDIFGLDLKREGGQYKLIPHNPNAGLSDL...,SEDLTDLKELTEESDLLSDIFGLDLKREGGQYKLIPHNPNAGLSDL...,CCGAGGATCTGACCGACTTAAAGGAGCTGACAGAGGAAAGTGACCT...,CCGAGGATCTGACCGACTTAAAGGAGCTGACAGAGGAAAGTGACCT...,+,Pathogenic,RYR2,C,G,237819181,1,SO:0001583|missense_variant
...,...,...,...,...,...,...,...,...,...,...,...,...
93,MGSVRTNRYSIVSSEEDGMKLATMAVANGFGNGKSKVHTRQQCRSR...,MGSVRTNRYSIVSSEEDGMKLATMAVANGFGNGKSKVHTRQQCRSR...,ATGGGCAGTGTGCGAACCAACCGCTACAGCATCGTCTCTTCAGAAG...,ATGGGCAGTGTGCGAACCAACCGCTACAGCATCGTCTCTTCAGAAG...,+,Pathogenic/Likely_pathogenic,KCNJ2,A,G,70175958,1,SO:0001583|missense_variant
94,MVVPEKEQSWIPKIFKKKTCTTFIVDSTDPGGTLCQCGRPRTAHPA...,MVVPEKEQSWIPKIFKKKTCTTFIVDSTDPGGTLCQCGRPRTAHPA...,ATGGTGGTGCCGGAGAAGGAGCAGAGCTGGATCCCCAAGATCTTCA...,ATGGTGGTGCCGGAGAAGGAGCAGAGCTGGATCCCCAAGATCTTCA...,+,Likely_benign,TRPM4,G,A,49182690,1,"SO:0001583|missense_variant,SO:0001623|5_prime..."
95,MCIIFFKFDPRPVSKNAYRLILAANRDEFYSRPSKLADFWGNNNEI...,MCIIFFKFDPRPVSKNAYRLILAANRDEFYSRPSKLADFWGNNNEI...,ATGTGCATCATCTTCTTTAAGTTTGATCCTCGCCCTGTTTCCAAAA...,ATGTGCATCATCTTCTTTAAGTTTGATCCTCGCCCTGTTTCCAAAA...,+,Pathogenic/Likely_pathogenic,TANGO2,C,T,20052575,1,"SO:0001583|missense_variant,SO:0001587|nonsens..."
96,MCIIFFKFDPRPVSKNAYRLILAANRDEFYSRPSKLADFWGNNNEI...,MCIIFFKFDPRPVSKNAYRLILAANRDEFYSRPSKLADFWGNNNEI...,ATGTGCATCATCTTCTTTAAGTTTGATCCTCGCCCTGTTTCCAAAA...,ATGTGCATCATCTTCTTTAAGTTTGATCCTCGCCCTGTTTCCAAAA...,+,Pathogenic,TANGO2,C,T,20052581,1,"SO:0001583|missense_variant,SO:0001587|nonsens..."


In [18]:
# Adjust the mapping to account for variations in the 'cln_significance' descriptions
def map_significance(value):
    value = value.lower()  # Convert to lower case to handle case variations uniformly
    if 'benign' in value:  # This will match "Benign" and "Likely_benign"
        return 0
    else:  # Matches "Pathogenic"
        return 1
#sequences_df_rc['seq_a'] = sequences_df_rc['seq_a'].str.upper()
#sequences_df_rc['seq_b'] = sequences_df_rc['seq_b'].str.upper()
# Apply the mapping function to the 'cln_significance' column
sequences_df['labels'] = sequences_df['label'].apply(map_significance)
#sequences_df_rc['POS'] = sequences_df['POS']
#sequences_df_reset = sequences_df_rc.reset_index(drop=True)

# Now sequences_df_reset contains the appended 'cln_significance_mapped' column
sequences_df_rc = sequences_df[['ref_prot','alt_prot','labels']]
sequences_df_rc = sequences_df_rc.rename(columns={'ref_prot': 'wt_seq', 'alt_prot': 'mut_seq'})
print(sequences_df_rc)

                                                wt_seq  \
0    MARGKAKEEGSWKKFIWNSEKKEFLGRTGGSWFKILLFYVIFYGCL...   
1    MADGGEGEDEIQFLRTDDEVVLQCTATIHKEQQKLCLAAEGFGNRL...   
2    SAVCALGNHRVAHALCSHVDEPQLLYAIENKYMPGLLRAGYYDLLI...   
3    QTGNNTTVNIIISTVDYLLRVQESISDFYWYYSGKDVIDEQGQRNF...   
4    ERVYFEISESSRTQWEKPQVKESKRQFIFDVVNEGGEKEKMELFVN...   
..                                                 ...   
880  MCIIFFKFDPRPVSKNAYRLILAANRDEFYSRPSKLADFWGNNNEI...   
881  MCIIFFKFDPRPVSKNAYRLILAANRDEFYSRPSKLADFWGNNNEI...   
882  MCIIFFKFDPRPVSKNAYRLILAANRDEFYSRPSKLADFWGNNNEI...   
883  MCIIFFKFDPRPVSKNAYRLILAANRDEFYSRPSKLADFWGNNNEI...   
884  MCIIFFKFDPRPVSKNAYRLILAANRDEFYSRPSKLADFWGNNNEI...   

                                               mut_seq  labels  
0    MARGKAKEEGSWKKFIWNSEKKEFLGRTGGSWFKILLFYVIFYGCL...       1  
1    MADGGEGEDEIQFLRTDDEVVLQCTATIHKEQQKLCLAAEGFGNRL...       1  
2    SAVCALGNHRVAHALCSHVDEPQLLYAIENKYMPGLLRAGYYDLLI...       1  
3    QTGNNTTVNIIISTVDYLLRVQESISDFYWYYSGKDVI

In [19]:
sequences_df_rc.to_csv('/common/zhangz2lab/zhanh/esm-variants/cropped/Clinvar_uncertain_ARM_protein.csv', index=False)

In [1]:
import pandas as pd
df = pd.read_csv('/common/zhangz2lab/zhanh/Propath-2/PLLR_ARM.csv')
value_counts = df['true_labels_callback'].value_counts()

# Print the counts
print(value_counts)

0    77
1    21
Name: true_labels_callback, dtype: int64


In [2]:
df = pd.read_csv('/common/zhangz2lab/zhanh/esm-variants/cropped/Clinvar_uncertain_ARM_protein.csv')
df

,wt_seq,mut_seq,labels
0,MARGKAKEEGSWKKFIWNSEKKEFLGRTGGSWFKILLFYVIFYGCL...,MARGKAKEEGSWKKFIWNSEKKEFLGRTGGSWFKILLFYVIFYGCL...,1
1,MADGGEGEDEIQFLRTDDEVVLQCTATIHKEQQKLCLAAEGFGNRL...,MADGGEGEDEIQFLRTDDEVVLQCTATIHKEQQKLCLAAEGFGNRL...,1
2,SAVCALGNHRVAHALCSHVDEPQLLYAIENKYMPGLLRAGYYDLLI...,SAVCALGNHRVAHALCSHVDEPQLLYAIENKYMPGLLRAGYYDLLI...,1
3,QTGNNTTVNIIISTVDYLLRVQESISDFYWYYSGKDVIDEQGQRNF...,QTGNNTTVNIIISTVDYLLRVQESISDFYWYYSGKDVIDEQGQRNF...,1
4,ERVYFEISESSRTQWEKPQVKESKRQFIFDVVNEGGEKEKMELFVN...,ERVYFEISESSRTQWEKPQVKESKRQFIFDVVNEGGEKEKMELFVN...,1
...,...,...,...
880,MCIIFFKFDPRPVSKNAYRLILAANRDEFYSRPSKLADFWGNNNEI...,MCIIFFKFDPRPVSKNAYRLILAANRDEFYSRPSKLADFWGNNNEI...,1
881,MCIIFFKFDPRPVSKNAYRLILAANRDEFYSRPSKLADFWGNNNEI...,MCIIFFKFDPRPVSKNAYRLILAANRDEFYSRPSKLADFWGNNNEI...,1
882,MCIIFFKFDPRPVSKNAYRLILAANRDEFYSRPSKLADFWGNNNEI...,MCIIFFKFDPRPVSKNAYRLILAANRDEFYSRPSKLADFWGNNNEI...,1
883,MCIIFFKFDPRPVSKNAYRLILAANRDEFYSRPSKLADFWGNNNEI...,MCIIFFKFDPRPVSKNAYRLILAANRDEFYSRPSKLADFWGNNNEI...,1
